In [23]:
from __future__ import print_function, division
import numpy as np
from chainconsumer import ChainConsumer
import astropy.io.fits as pyfits
from scipy.special import erfinv

In [24]:
def priortransform(cube):
        # definition of the parameter width, by transforming from a unit cube - switched redshift to be the last index
        # skip this, emu on unit cube
        #cube = cube.copy()
        cube[0] = cube[0] * 5.5 + 7.
        ##cube[0] = 10**(cube[0] * 5.5 - 3) # basically, this sets bounds on the mass: 7 to 12.5 -> -3 to 2 (A=1 -> 10^10)
        cube[1] = cube[1] * 10 # mu: 0 to 10
        cube[2] = cube[2] * 4.8 + 0.2 # sigma: 0.2 to 5 
        ##cube[3] = cube[3] * 10 # z: 0 to 10
        cube[3] = 10**(0.3 * erfinv(2.*(cube[3]-0.5)))  # metallicity scatter: Gaussian about 0, sigma = 0.3 dex - pinch this at some limiting values - 0.001, 0.999??
        cube[4] = cube[4] * 2.  # dust extinction (doubled in birth clouds): 0 to 2.
        cube[5] = 10**(cube[5]) - 1. # log-spaced redshift sampling, z: 0 to 9.
        ##cube[5] = cube[5] # z: 0 to 9. The log spacing is set earlier, so we just output the same value
        return cube


In [25]:
data = pyfits.open('tmp_mcmc_GPflow.fits')[1].data[2000:]
chain = np.vstack((data['Mass'],data['mu'],data['sigma'],data['metallicity'],data['dust'],data['z'])).T

In [26]:
chain.shape

(7000, 6)

In [27]:
# convert unit cube into physical values
for i in range(chain.shape[0]):
    chain[i,:] = priortransform(chain[i,:])

In [28]:
c = ChainConsumer().add_chain(chain, parameters=["$Mass$", "$mu$", "$sigma$", "$metallicity$", "$dust$", "$z$"])

In [34]:
c.configure(kde=True, shade_alpha=0.1, flip=False)
#fig = c.plotter.plot_distributions()
fig = c.plotter.plot(extents=[(7, 12.5), (0, 10), (0.2, 5), (0., 3.), (0., 2.), (0., 9.)])

/home/will/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
/home/will/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


RuntimeError: libpng signaled error

<Figure size 648x648 with 36 Axes>

In [13]:
#from numpy.random import normal, multivariate_normal

In [15]:
#np.random.seed(0)
#cov = 1e2 * normal(size=(3, 3))
#rdata = multivariate_normal(1e3 * normal(size=3), np.dot(cov, cov.T), size=100000)
#rdata.shape

(100000, 3)